#   **Projet Maths Info Horsin/Cattan**





Soit $f :  \mathbb{R^2} \to \mathbb{R}$ et $c \in \mathbb{R}$.

Le but du projet est de trouver l'ensemble $\{ \:(x,y) \:/ \:f(x,y)=c \: \}$.

# 1ère Partie: **Contour Simple**

## L'amorce

Nous avons choisi d'utiliser un algorithme dichotomique dans un premier temps, qui est plus robuste qu'un algorithme de Newton. En effet, n'ayant aucune information sur la valeur initiale (indispensable pour garantir la convergence), il semble plus cohérent d'utiliser une dichotomie.
 

In [9]:
def find_seed(f,c=0.0,x=0.0,eps=2**(-26)):
    if not f(x,1.0)<= c <= f(x,0.0) and not f(x,0)<=c<=f(x,1.0):
            return None  
    a=0
    b=1
    t=0.5
    while abs(f(x,t)-c)>eps:  
        if (f(x,a)-c)*(f(x,t)-c)>=0:
            a=t
        else :
            b=t
        t=(a+b)/2
    return t 

### Le contour simple

Pour la suite, nous allons définir quelques fonctions dont nous aurons besoin pour la fonction simple_contour.
Commençons par importer les bibliothèques utiles.

In [10]:
import matplotlib.pyplot as plt  
import autograd
from autograd import numpy as np

Définissons maintenant une fonction permettant de calculer une distance euclidienne dans $\mathbb{R^2}$

In [11]:
def distanceeucl(x,y):
    return np.sqrt(((x[0]-y[0])**2)+((x[1]-y[1])**2))

Définissons ensuite une fonction permettant de donner à un vecteur une norme souhaitée.

In [12]:
def normalisation(vecteur,normev):
    norme=distanceeucl(vecteur,(0,0))
    x=vecteur[0]
    y=vecteur[1]
    return (x*normev/norme,y*normev/norme)

Définissons enfin la fonction gradient qui calcule le gradient à l'aide de la bibliothèque autograd. 

In [13]:
def grad(f,x,y):
    g=autograd.grad
    return np.r_[g(f,0)(x,y),g(f,1)(x,y)]

Nous pouvons maintenant définir la fonction simple_contour qui blablabla faut expliquer et commenter le programme

In [20]:
def simple_contour(f,c=0.0,delta=0.01):
    x=[0.0]  #on définit un premier point sur l'arête gauche à l'aide de find_seed
    y=[find_seed(f,c,0.0)] 
    avant=[x[-1],y[-1]]
    gradient=grad(f,avant[0],avant[1]) #on calcule le gradient au point précédemment trouvé
    tang=[-gradient[1],gradient[0]]
    v=normalisation(tang,delta)  #on obtient la tangente à la courbe de niveau 
    if v[0] >= 0 :  #on vérifie que le point choisi en second partira bien vers la droite (x>=0)
        p1=(avant[0]+v[0],avant[1]+v[1])
    else : 
        p1=(avant[0]-v[0],avant[1]-v[1])
    
    def F(x,y):
        return np.array([f(x,y)-c,(x-avant[0])**2+(y-avant[1])**2-delta**2])
        
    def Jacob(F,x,y):                #on définit la jacobienne.
        j = autograd.jacobian
        return np.c_[j(F,0)(x,y),j(F,1)(x,y)]
    
    X=np.array(p1)
    A=F(p1[0],p1[1])   
    while distanceeucl(A,[0,0]) >= 2**(-10) :
        A=F(X[0],X[1])
        X=X-np.linalg.inv(Jacob(F,X[0],X[1])).dot(np.array(A))
    
    x+=[X[0]]
    y+=[X[1]]
    while x[-1]<1-delta and y[-1]>0+delta and y[-1]<1-delta:
        distance=[]
        avant=[x[-1],y[-1]]
        gradient=grad(f,avant[0],avant[1])
        tang=[-gradient[1],gradient[0]]
        v=normalisation(tang,delta)
        p1=[avant[0]+v[0],avant[1]+v[1]]
        p2=[avant[0]-v[0],avant[1]-v[1]]
        p=[p1,p2]
        for i in p:
            distance+=[distanceeucl((x[-2],y[-2]),i)]
        e=distance.index(max(distance))
        
        def F(x,y):
            return np.array([f(x,y)-c,(x-avant[0])**2+(y-avant[1])**2-delta**2])
        
        def Jacob(F,x,y):                #on définit la jacobienne.
            j = autograd.jacobian
            return np.c_[j(F,0)(x,y),j(F,1)(x,y)]
       
        X=np.array(p[e])
        A=F(X[0],X[1])   # Il s'agit du point intermédiaire permettant d'initialiser Newton.
        while distanceeucl(A,[0,0]) >= 2**(-10) :
            A=F(X[0],X[1])
            X=X-np.linalg.inv(Jacob(F,X[0],X[1])).dot(np.array(A))
        x+=[X[0]]
        y+=[X[1]]
    return [x,y]

On peut maintenant définir des fonctions tests, et tester ce premier programme sur ces fonctions tests. 

In [ ]:
def f(x,y):
    return 2*(np.exp(-x**2-y**2)-np.exp(-(x-1)**2-(y-1)**2))

def h(x,y):
    return x**2+(y-0.4)**2

In [ ]:
c=input("Donnez la valeur du réel c >> ")    
data=simple_contour(h,float(c))
print(data)
plt.plot(data[0],data[1])
plt.grid()
plt.title(f"Courbe de niveau pour c={float(c)}")
plt.xlabel("x")
plt.ylabel("y")
plt.show()

# 2ème partie : Contour complexe